# EE4375-2022: Sixth Lab Session: Galerkin Finite Element Method for the Poisson Equation on the Unit Square 

 Solves the Poisson equation $- \bigtriangleup u = f$ on the unit square domain $\Omega=(0.,1.0)^2$ supplied with the homogeneous Dirichlet boundary condition $u = 0$ on $\Gamma$. The computational domain is kept simple on purpose for illustration purposes. The source function $f$ is defined as the sourcefct in what follows. Linear finite elements on triangular elements are used. The mesh is generated using the mesh generation software called gmsh (see [Gmsh](https://gmsh.info)). More information on the use of GMSH is provided at [extended-lab-sessions/gmsh](https://github.com/ziolai/finite_element_electrical_engineering/tree/main/extended-lab-sessions/gmsh).

## Import Packages

In [2]:
import Gmsh: gmsh 
using GR 
using LinearAlgebra
using SparseArrays 
using Plots
using LaTeXStrings

## Section 1: Geometry and Mesh Generation of Unit Square Geometry 
The code that follows performs <b>geometry generation</b> and <b>mesh generation</b> on the unit square geometry. 

First the <b>geometry</b> is generated in the following five steps:
1. four corner points of the square are defined. The points are labeled as 1 through 4;
2. four lines are defined as the edges of the square are defined by connecting previously defined points. Edges are formed by connecting points pairwise. The lines are given a start and end point. The lines are thus oriented. The edges are labeled as 1 through 4;  
3. the boundary of the square is defined by a loop connecting the four edges. The orientation of the edges given an orientation to the loop. The loop is oriented such that an imaginary observer walking on the loop finds the domain on his left-hand side. The loop is labeled as 1;  
4. the surface of the square is defined by the loop. It is on this square that the mesh generation will take place. This square is labeled as 1; 
5. a name is given to the surface; 

Next the <b>mesh</b> on the geometry is defined by mesh generation. The mesh is generated by first meshing the four edges of the square. The mesh is subsequently propagated towards the interior of the square. The mesh density is controlled by the parameter lc. 

The mesh is optionally written to file and visualized using the GUI. 

<b>Exercises </b>: 
1. change coordinate of one of the points and regenerate the mesh;
2. change the mesh density by changing the value of the parameter lc and regenerate the mesh. Apply different mesh density on one or more point or edges of the square; 
3. extend the code to the generation of a mesh on a pentagon; 
4. extend the code to the generation of a mesh on an L-shaped domain; 
5. extend the code to the generation of a mesh on a square with an inner square removed this practising the orientation of the loops; 
6. share imagines of the meshes obtained of social media of choice and notify your favorite finite element instructor; 

In [3]:
#..1/4: initialize gmsh 
gmsh.initialize()

#..2/4: generate geometry 
gmsh.option.setNumber("General.Terminal", 1)
gmsh.model.add("square")
#..set mesh density parameter 
lc = .2e-1
#..define four points via (x,y,z) coordinates 
gmsh.model.geo.addPoint(0, 0, 0, lc, 1)
gmsh.model.geo.addPoint(1., 0,  0, lc, 2)
gmsh.model.geo.addPoint(1., 1., 0, lc, 3)
gmsh.model.geo.addPoint(0, 1., 0, lc, 4)
#..define four edges by connecting point labels pairwise  
gmsh.model.geo.addLine(1, 2, 1)
gmsh.model.geo.addLine(2, 3, 2)
gmsh.model.geo.addLine(3, 4, 3)
gmsh.model.geo.addLine(4, 1, 4)
#..define curved loop by connecting four edge labels  
gmsh.model.geo.addCurveLoop([1, 2, 3, 4], 1)
#..define surface by curved loop 
gmsh.model.geo.addPlaneSurface([1], 1)
#..give label to the surface 
#..syntax of the function being used is gmsh.model.setPhysicalName(dim, tag, name) 
gmsh.model.setPhysicalName(2, 1, "My surface")
#..synchronize model 
gmsh.model.geo.synchronize()

#..3/4: generate two-dimensional mesh 
gmsh.model.mesh.generate(2)
#..if true, write mesh to file for further processing 
if (true) gmsh.write("square.msh") end 
#..if true, visualize mesh through the GUI 
if (false) gmsh.fltk.run() end 

#..4/4: finalize gmsh 
gmsh.finalize()

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 30%] Meshing curve 2 (Line)
Info    : [ 50%] Meshing curve 3 (Line)
Info    : [ 80%] Meshing curve 4 (Line)
Info    : Done meshing 1D (Wall 0.00117042s, CPU 0.000219s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.0525324s, CPU 0.049817s)
Info    : 3014 nodes 6030 elements
Info    : Writing 'square.msh'...
Info    : Done writing 'square.msh'


## Section 2:/ Read Mesh from file and perform loop over the elements 
Exercise: 
- verify that sum of area of elements over mesh equals area of entire 2D domain;

In [4]:
#..1/4: Finalize gmsh
gmsh.initialize()

#..2/4: Read mesh from file
gmsh.open("square.msh")

#..3/4: perform loop over the elemements 
element_types, element_ids, element_connectivity = gmsh.model.mesh.getElements(2)
nelements = length(element_ids[1])
  
for element_id in 1:nelements

    #....retrieve global numbering of the local nodes of the current element
    node1_id = element_connectivity[1][3*(element_id-1)+1]
    node2_id = element_connectivity[1][3*(element_id-1)+2]
    node3_id = element_connectivity[1][3*(element_id-1)+3]

    println("on element ", element_id, " node-1 has global number ", node1_id)
    println("on element ", element_id, " node-2 has global number ", node2_id)
    println("on element ", element_id, " node-3 has global number ", node3_id)
    println(" ")

end 

#..4/4: Finalize gmsh
gmsh.finalize()

Info    : Reading 'square.msh'...
Info    : 9 entities
Info    : 3014 nodes
Info    : 6030 elements
Info    : Done reading 'square.msh'
on element 1 node-1 has global number 2886
on element 1 node-2 has global number 1735
on element 1 node-3 has global number 2998
 
on element 2 node-1 has global number 1560
on element 2 node-2 has global number 1921
on element 2 node-3 has global number 1946
 
on element 3 node-1 has global number 1857
on element 3 node-2 has global number 235
on element 3 node-3 has global number 2024
 
on element 4 node-1 has global number 226
on element 4 node-2 has global number 2886
on element 4 node-3 has global number 2998
 
on element 5 node-1 has global number 2884
on element 5 node-2 has global number 2257
on element 5 node-3 has global number 2999
 
on element 6 node-1 has global number 2053
on element 6 node-2 has global number 1343
on element 6 node-3 has global number 2512
 
on element 7 node-1 has global number 2024
on element 7 node-2 has global number

on element 77 node-1 has global number 166
on element 77 node-2 has global number 167
on element 77 node-3 has global number 1764
 
on element 78 node-1 has global number 192
on element 78 node-2 has global number 193
on element 78 node-3 has global number 1765
 
on element 79 node-1 has global number 73
on element 79 node-2 has global number 74
on element 79 node-3 has global number 2143
 
on element 80 node-1 has global number 250
on element 80 node-2 has global number 1988
on element 80 node-3 has global number 1989
 
on element 81 node-1 has global number 1962
on element 81 node-2 has global number 251
on element 81 node-3 has global number 1964
 
on element 82 node-1 has global number 80
on element 82 node-2 has global number 81
on element 82 node-3 has global number 1767
 
on element 83 node-1 has global number 2894
on element 83 node-2 has global number 1318
on element 83 node-3 has global number 2903
 
on element 84 node-1 has global number 227
on element 84 node-2 has global n

on element 152 node-3 has global number 2964
 
on element 153 node-1 has global number 2858
on element 153 node-2 has global number 2107
on element 153 node-3 has global number 2980
 
on element 154 node-1 has global number 1995
on element 154 node-2 has global number 1193
on element 154 node-3 has global number 2885
 
on element 155 node-1 has global number 1994
on element 155 node-2 has global number 1993
on element 155 node-3 has global number 2881
 
on element 156 node-1 has global number 1990
on element 156 node-2 has global number 1989
on element 156 node-3 has global number 1991
 
on element 157 node-1 has global number 1762
on element 157 node-2 has global number 1763
on element 157 node-3 has global number 1768
 
on element 158 node-1 has global number 266
on element 158 node-2 has global number 1760
on element 158 node-3 has global number 2991
 
on element 159 node-1 has global number 2897
on element 159 node-2 has global number 1731
on element 159 node-3 has global number 28

on element 227 node-3 has global number 2732
 
on element 228 node-1 has global number 1614
on element 228 node-2 has global number 2034
on element 228 node-3 has global number 2801
 
on element 229 node-1 has global number 225
on element 229 node-2 has global number 1770
on element 229 node-3 has global number 2845
 
on element 230 node-1 has global number 686
on element 230 node-2 has global number 684
on element 230 node-3 has global number 2849
 
on element 231 node-1 has global number 727
on element 231 node-2 has global number 725
on element 231 node-3 has global number 2199
 
on element 232 node-1 has global number 688
on element 232 node-2 has global number 690
on element 232 node-3 has global number 1832
 
on element 233 node-1 has global number 2845
on element 233 node-2 has global number 1770
on element 233 node-3 has global number 2848
 
on element 234 node-1 has global number 225
on element 234 node-2 has global number 2820
on element 234 node-3 has global number 2834
 
on

on element 303 node-1 has global number 233
on element 303 node-2 has global number 1798
on element 303 node-3 has global number 2721
 
on element 304 node-1 has global number 691
on element 304 node-2 has global number 1780
on element 304 node-3 has global number 1882
 
on element 305 node-1 has global number 691
on element 305 node-2 has global number 688
on element 305 node-3 has global number 1832
 
on element 306 node-1 has global number 1780
on element 306 node-2 has global number 691
on element 306 node-3 has global number 1832
 
on element 307 node-1 has global number 1388
on element 307 node-2 has global number 1387
on element 307 node-3 has global number 1845
 
on element 308 node-1 has global number 1256
on element 308 node-2 has global number 2116
on element 308 node-3 has global number 2117
 
on element 309 node-1 has global number 707
on element 309 node-2 has global number 704
on element 309 node-3 has global number 1858
 
on element 310 node-1 has global number 703
on e

...

on element 3578 node-1 has global number 1490
on element 3578 node-2 has global number 1678
on element 3578 node-3 has global number 1688
 
on element 3579 node-1 has global number 2402
on element 3579 node-2 has global number 305
on element 3579 node-3 has global number 2439
 
on element 3580 node-1 has global number 335
on element 3580 node-2 has global number 334
on element 3580 node-3 has global number 336
 
on element 3581 node-1 has global number 1002
on element 3581 node-2 has global number 650
on element 3581 node-3 has global number 1003
 
on element 3582 node-1 has global number 116
on element 3582 node-2 has global number 117
on element 3582 node-3 has global number 213
 
on element 3583 node-1 has global number 1275
on element 3583 node-2 has global number 1138
on element 3583 node-3 has global number 1375
 
on element 3584 node-1 has global number 2486
on element 3584 node-2 has global number 1185
on element 3584 node-3 has global number 2505
 
on element 3585 node-1 has g

on element 3651 node-3 has global number 2578
 
on element 3652 node-1 has global number 420
on element 3652 node-2 has global number 419
on element 3652 node-3 has global number 421
 
on element 3653 node-1 has global number 1864
on element 3653 node-2 has global number 1862
on element 3653 node-3 has global number 1865
 
on element 3654 node-1 has global number 114
on element 3654 node-2 has global number 115
on element 3654 node-3 has global number 269
 
on element 3655 node-1 has global number 115
on element 3655 node-2 has global number 116
on element 3655 node-3 has global number 2427
 
on element 3656 node-1 has global number 1659
on element 3656 node-2 has global number 910
on element 3656 node-3 has global number 2402
 
on element 3657 node-1 has global number 960
on element 3657 node-2 has global number 952
on element 3657 node-3 has global number 2347
 
on element 3658 node-1 has global number 945
on element 3658 node-2 has global number 1099
on element 3658 node-3 has globa

on element 3725 node-3 has global number 2460
 
on element 3726 node-1 has global number 884
on element 3726 node-2 has global number 883
on element 3726 node-3 has global number 886
 
on element 3727 node-1 has global number 1554
on element 3727 node-2 has global number 1555
on element 3727 node-3 has global number 1556
 
on element 3728 node-1 has global number 1682
on element 3728 node-2 has global number 1322
on element 3728 node-3 has global number 2430
 
on element 3729 node-1 has global number 242
on element 3729 node-2 has global number 27
on element 3729 node-3 has global number 2472
 
on element 3730 node-1 has global number 1844
on element 3730 node-2 has global number 1574
on element 3730 node-3 has global number 1924
 
on element 3731 node-1 has global number 1826
on element 3731 node-2 has global number 1325
on element 3731 node-3 has global number 2421
 
on element 3732 node-1 has global number 1014
on element 3732 node-2 has global number 1013
on element 3732 node-3 has

Excessive output truncated after 524288 bytes.

1436
on element 3799

## Section 3:/ Extract Data from the Mesh and Perform a FEM Computation  
This version of the code performs a single loop over the elements. The loop over the degrees of freedom inside the loop over the elements is avoided. 

Exercises: 
1. study the assembly of the vector;
2. study the assembly of the matrix; 

In [5]:
function area_triangle(x1,x2,x3,y1,y2,y3)
    x12 = x2 - x1; x13 = x3-x1;
    y12 = y2 - y1; y13 = y3-y1;
    area_id = x12*y13 - x13*y12; 
    area_id = abs(area_id)/2
    return area_id 
end

area_triangle (generic function with 1 method)

In [9]:
#..1/12: Finalize gmsh
gmsh.initialize()

#..2/12: Generate the mesh
gmsh.open("square.msh")

#..3/12 Get and sort the mesh nodes
#..Observe that although the mesh is two-dimensional,
#..the z-coordinate that is equal to zero is stored as well.
#..Observe that the coordinates are stored contiguously for computational
#..efficiency
node_ids, node_coord, _ = gmsh.model.mesh.getNodes()
nnodes = length(node_ids)
#..sort the node coordinates by ID, such that Node one sits at row 1
tosort = [node_ids node_coord[1:3:end] node_coord[2:3:end]];
sorted = sortslices(tosort , dims = 1);
node_ids = sorted[:,1]
xnode = sorted[:,2]
ynode = sorted[:,3]

#..4/12 Get the mesh elements
#..observe that we get all the two-dimensional triangular elements from the mesh
element_types, element_ids, element_connectivity = gmsh.model.mesh.getElements(2)
nelements = length(element_ids[1])

#..5/12 Define the source function and apply the source function to the local coordinates
sourcefunction(x,y) = x+y
fvalues = map(sourcefunction, xnode, ynode)

#..6/12 Initialize global matrix A and global vector f
#..observe that for simplicity we use dense matrix here
A = zeros(nnodes,nnodes)
M = zeros(nnodes,nnodes)
f = zeros(nnodes,1)

#..7/12 Perform a loop over the elements
for element_id in 1:nelements

  #....retrieve global numbering of the local nodes of the current element
  node1_id = element_connectivity[1][3*(element_id-1)+1]
  node2_id = element_connectivity[1][3*(element_id-1)+2]
  node3_id = element_connectivity[1][3*(element_id-1)+3]
  I = Vector{Int64}([node1_id; node2_id; node3_id])

  #....retrieve the x and y coordinates of the local nodes of the current element
  xnode1 = xnode[node1_id]; xnode2 = xnode[node2_id]; xnode3 = xnode[node3_id];
  ynode1 = ynode[node1_id]; ynode2 = ynode[node2_id]; ynode3 = ynode[node3_id];

  #....compute surface area of the current element
  area_id = area_triangle(xnode1,xnode2,xnode3,ynode1,ynode2,ynode3)

  #....if desired, print global number numbers and element area to screen
  if (false)
    println("on element ", element_id, " node-1 has global number ", node1_id)
    println("on element ", element_id, " node-2 has global number ", node2_id)
    println("on element ", element_id, " node-3 has global number ", node3_id)
    println("on element ", element_id, " area = ", area_id)
    println(" ")
  end
    
  #....compute local vector contribution floc of the current element
  floc = area_id/3*fvalues[I]

  #....compute local matrix contribution Aloc of the current element
  Emat = [[xnode1;xnode2;xnode3] [ynode1;ynode2;ynode3] [1;1;1]]\UniformScaling(1.);
  Emat[3,:] .= 0;
  Aloc = area_id*(transpose(Emat)*Emat);
  Mloc = area_id/3*[1 0 0; 0 1 0; 0 0 1]; 
    
  #....perform loop over nodes of the current element
  #....and add local contribution Aloc to global matrix entity A
  f[I] += floc
  A[I,I] += Aloc
  M[I,I] += Mloc

end

#..8/12 Handle the boundary conditions
#..retrieve boundary nodes by loop over corner point and boundary edges
node_ids1=[]; node_ids2=[]; node_ids3=[]; node_ids4=[]; 
node_ids5=[]; node_ids6=[]; node_ids7=[]; node_ids8=[]; 
node_ids1, node_coord, _ = gmsh.model.mesh.getNodes(0,1)
node_ids2, node_coord, _ = gmsh.model.mesh.getNodes(0,2)
node_ids3, node_coord, _ = gmsh.model.mesh.getNodes(0,3)
node_ids4, node_coord, _ = gmsh.model.mesh.getNodes(0,4)
node_ids5, node_coord, _ = gmsh.model.mesh.getNodes(1,1)
node_ids6, node_coord, _ = gmsh.model.mesh.getNodes(1,2)
node_ids7, node_coord, _ = gmsh.model.mesh.getNodes(1,3)
node_ids8, node_coord, _ = gmsh.model.mesh.getNodes(1,4)
bnd_node_ids = union(node_ids1,node_ids2,node_ids3,node_ids4,node_ids5,node_ids6,node_ids7,node_ids8)
A[bnd_node_ids,:] .= 0;
A[bnd_node_ids,bnd_node_ids] = Diagonal(ones(size(bnd_node_ids)))
f[bnd_node_ids] .= 0;

#..9/12 Make A sparse and M diagonal 
A = sparse(A)
M = Diagonal(M)

#..10/12 Compute the numerical solution
u = A\f

#..10/12: Finalize gmsh
gmsh.finalize()

#..11/12 Plot the numerical solution
GR.trisurf(xnode,ynode,u)

Info    : Reading 'square.msh'...
Info    : 9 entities
Info    : 3014 nodes
Info    : 6030 elements
Info    : Done reading 'square.msh'


## Section 4:/ Magnetic Field by Coil  

In [10]:
#..1/12: Finalize gmsh
gmsh.initialize()

#..2/12: Generate the mesh
gmsh.open("square.msh")

#..3/12 Get and sort the mesh nodes
#..Observe that although the mesh is two-dimensional,
#..the z-coordinate that is equal to zero is stored as well.
#..Observe that the coordinates are stored contiguously for computational
#..efficiency
node_ids, node_coord, _ = gmsh.model.mesh.getNodes()
nnodes = length(node_ids)
#..sort the node coordinates by ID, such that Node one sits at row 1
tosort = [node_ids node_coord[1:3:end] node_coord[2:3:end]];
sorted = sortslices(tosort , dims = 1);
node_ids = sorted[:,1]
xnode = sorted[:,2]
ynode = sorted[:,3]

#..4/12 Get the mesh elements
#..observe that we get all the two-dimensional triangular elements from the mesh
element_types, element_ids, element_connectivity = gmsh.model.mesh.getElements(2)
nelements = length(element_ids[1])

#..5/12 Define the source function and apply the source function to the local coordinates
#sourcefunction(x,y) = x+y
sourcefunction(x,y) = (x>0.3)*(x<0.7)*(y>0.7)*(y<0.9) - (x>0.3)*(x<0.7)*(y>0.1)*(y<0.3) 
fvalues = map(sourcefunction, xnode, ynode)

#..6/12 Initialize global matrix A and global vector f
#..observe that for simplicity we use dense matrix here
A = zeros(nnodes,nnodes)
M = zeros(nnodes,nnodes)
f = zeros(nnodes,1)

#..7/12 Perform a loop over the elements
for element_id in 1:nelements

  #....retrieve global numbering of the local nodes of the current element
  node1_id = element_connectivity[1][3*(element_id-1)+1]
  node2_id = element_connectivity[1][3*(element_id-1)+2]
  node3_id = element_connectivity[1][3*(element_id-1)+3]
  I = Vector{Int64}([node1_id; node2_id; node3_id])

  #....retrieve the x and y coordinates of the local nodes of the current element
  xnode1 = xnode[node1_id]; xnode2 = xnode[node2_id]; xnode3 = xnode[node3_id];
  ynode1 = ynode[node1_id]; ynode2 = ynode[node2_id]; ynode3 = ynode[node3_id];

  #....compute surface area of the current element
  area_id = area_triangle(xnode1,xnode2,xnode3,ynode1,ynode2,ynode3)

  #....if desired, print global number numbers and element area to screen
  if (false)
    println("on element ", element_id, " node-1 has global number ", node1_id)
    println("on element ", element_id, " node-2 has global number ", node2_id)
    println("on element ", element_id, " node-3 has global number ", node3_id)
    println("on element ", element_id, " area = ", area_id)
    println(" ")
  end

  #....compute local vector contribution floc of the current element
  floc = area_id/3*fvalues[I]

  #....compute local matrix contribution Aloc of the current element
  Emat = [[xnode1;xnode2;xnode3] [ynode1;ynode2;ynode3] [1;1;1]]\UniformScaling(1.);
  Emat[3,:] .= 0;
  Aloc = area_id*(transpose(Emat)*Emat);
  Mloc = area_id/3*[1 0 0; 0 1 0; 0 0 1]; 
    
  #....perform loop over nodes of the current element
  #....and add local contribution Aloc to global matrix entity A
  f[I] += floc  
  A[I,I] += Aloc
  M[I,I] += Mloc

end

#..8/12 Handle the boundary conditions
#..retrieve boundary nodes by loop over corner point and boundary edges
node_ids1=[]; node_ids2=[]; node_ids3=[]; node_ids4=[]; 
node_ids5=[]; node_ids6=[]; node_ids7=[]; node_ids8=[]; 
node_ids1, node_coord, _ = gmsh.model.mesh.getNodes(0,1)
node_ids2, node_coord, _ = gmsh.model.mesh.getNodes(0,2)
node_ids3, node_coord, _ = gmsh.model.mesh.getNodes(0,3)
node_ids4, node_coord, _ = gmsh.model.mesh.getNodes(0,4)
node_ids5, node_coord, _ = gmsh.model.mesh.getNodes(1,1)
node_ids6, node_coord, _ = gmsh.model.mesh.getNodes(1,2)
node_ids7, node_coord, _ = gmsh.model.mesh.getNodes(1,3)
node_ids8, node_coord, _ = gmsh.model.mesh.getNodes(1,4)
bnd_node_ids = union(node_ids1,node_ids2,node_ids3,node_ids4,node_ids5,node_ids6,node_ids7,node_ids8)
A[bnd_node_ids,:] .= 0;
A[bnd_node_ids,bnd_node_ids] = Diagonal(ones(size(bnd_node_ids)))
f[bnd_node_ids] .= 0;

#..9/12 Make A sparse and M diagonal 
A = sparse(A)
M = Diagonal(M)

#..9/12 Compute the numerical solution
u = A\f

#..10/12: Finalize gmsh
gmsh.finalize()

#..11/11 Plot the source term and the numerical solution
GR.subplot(1,2,1)
p1 = GR.tricont(xnode,ynode,f)
GR.subplot(1,2,2)
p1 = GR.tricont(xnode,ynode,u)

Info    : Reading 'square.msh'...
Info    : 9 entities
Info    : 3014 nodes
Info    : 6030 elements
Info    : Done reading 'square.msh'


## Section 5:/ Solving the Scalar Diffusion Equation in Time Domain 

In [23]:
using DifferentialEquations

### Defining the Problem 

In [ ]:
#..define the right-hand side of the ordinary differential equation of the equation of motion 
function scalarDiffusionEquation!(du,u,p,t)
    du .= M \ (sin(2*π*50*t)*f - A*u)
end

#..set initial position and velocity
u0 = fill(1.,nnodes)
                                    
#..set time begin and end forward
t0 = 0.0
tf = 2.0
tspan = (t0,tf)

#..define ODE problem to be solved  
prob_scalarDiffusion = ODEProblem(scalarDiffusionEquation!, u0, tspan)

#..solve ODE problem 
sol = DifferentialEquations.solve(prob_scalarDiffusion);

### Plotting the computed solution 

In [ ]:
dt = 2
tvec = Vector(t0:dt:tf)

Plots.plot(sol)
for i in 2:nnodes-1
    Plots.plot!(sol,vars=i)
end
p1 = Plots.plot!(sol)
title!("Displacement")

#..plot solution of velocity and position as function of time  
Plots.plot(p1)

In [ ]:
anim = @animate for i in t0:tf
    Plots.surface(xnode,ynode,sol(i/10),st=:surface)
    zlims!(-.2, 1.5)
end
gif(anim, "my-animation.gif", fps = 1)

## Section 6:/ Solving a Modified Scalar Diffusion Equation in Time Domain 

In [ ]:
# set damping matrix
C = .005*M 

#..define the right-hand side of the ordinary differential equation of the equation of motion 
function scalarWaveEquation!(ddu,du,u,p,t)
    ddu .= M \ (f - A*u - C*du)
end

#..set initial position and velocity
u0 = fill(0.,nnodes)

v0 = zeros(nnodes)
# v0[4] = 1
                                    
#..set time begin and end forward
t0 = 0.0
tf = 20.0
tspan = (t0,tf)           

#..define ODE problem to be solved  
prob = SecondOrderODEProblem(scalarWaveEquation!,v0,u0,tspan)

#..solve ODE problem 
sol = DifferentialEquations.solve(prob);

### Post-plotting the Solution using time-traces  

In [ ]:
dt = 1
tvec = Vector(t0:dt:tf)

#..velocity and position have vars=(1:N) and vars=(N+1,2N), respectively. 
Plots.plot(sol,vars=1)
for i in 2:nnodes-1
    Plots.plot!(sol,vars=i)
end
p1 = Plots.plot!(sol,vars=nnodes)
title!("Velocity")

plot(sol,vars=nnodes+1)
for i in nnodes+2:2*nnodes-1
    Plots.plot!(sol,vars=i)
end
p2 = Plots.plot!(sol,vars=2*nnodes)
title!("Displacement")

#..plot solution of velocity and position as function of time  
Plots.plot(p1,p2,layout=(2,1))

### Post-plotting using animation 

In [ ]:
anim = @animate for i in t0:tf
    Plots.surface(xnode,ynode,sol(i/10)[nnodes+1:2*nnodes],st=:surface)
    zlims!(-.2, .4)
end
gif(anim, "my-animation.gif", fps = 15)

## Section 7:/ Further Improvements/Extensions to this Notebook 
1. avoid matrix $A$ to be dense. Instead, declare $A$ to be sparse (without pre-allocating memory for $A$). Traverse over elements, assemble I, J, Avals lists. After traversal, use sparse to construct the $A$ matrix (see 1D FEM code). Implement traversal using both for loop and map; 
2. put assembly inside function. Test function for type stability. Profile assemby of matrix and vector for various problem sizes. Check whether making Aloc to be a static matrix has any advantages. Check whether updating lists for I, J and Avals in place has any advanges;  
3. idem for solve. Replace backslash by iterative solution method (AMG as preconditioner for CG) and profile again;  
4. create seperate function to generate Aloc, Mloc and floc allowing to introduce quadrature on the element; 
5. create assembly on quadrilateral elements; 
6. create assembly for second order triangular (quadrilateral) elements; 
7. create assembly for non-homogeneous Neumann boundary conditions; 
8. create assembly for three-dimensional geometries; 